# Testing EfficientNet 

In [ ]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
from tuning import callbacks as cb
import datetime
import os
import numpy as np

In [ ]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

image_size = (224,224)
batch_size = 32
epochs = 30
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", image_size, batch_size, preprocess_input)
(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace", image_size, batch_size, preprocess_input)


## Find learning rate

In [ ]:
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (224,224), "Efficient")
    metric_list = m.metrics_list()

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), 
                           loss="binary_crossentropy", 
                           metrics="accuracy")

    
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
ds_train = ds_train.with_options(options)
ds_val = ds_val.with_options(options)
ds_test = ds_test.with_options(options)

In [ ]:
epochs = 200
train_input = []
train_output = []
nr_batches = 10

model.fit(ds_train, steps_per_epoch=nr_batches, callbacks=[cb.LerningRateCallback(100, nr_batches)], epochs=epochs, verbose=2)

## Cross Validation EfficientNet without Transfer Learning (unweighted)

In [ ]:
preprocess = tf.keras.applications.efficientnet.preprocess_input
history = ev.kfoldCrossValidation("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", m.build_model(False, (224,224), "Efficient"), preprocess)

## Test EfficientNet without Transfer Learning (unweighted) 

In [ ]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

image_size = (224,224)
batch_size = 8
epochs = 60
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (224,224), "Efficient")
    metric_list = m.metrics_list()

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), 
                           loss="binary_crossentropy", 
                           metrics="accuracy")


utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_unweighted"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_unweighted.h5")

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_unweighted"
os.mkdir(dir_name)
    

test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)